# RAG Hybrydowy z Weaviate

W tym notebooku zaimplementujemy hybrydowy system RAG (Retrieval-Augmented Generation) używając Weaviate jako bazy wektorowej oraz wyszukiwarki opartej o BM25. Wykorzystamy dane z ulotek lekarstw znajdujące się w folderze "008-02. RAG-data/drug-data".

W tym przypadku hybrydowy RAG łączy wyszukiwanie wektorowe z wyszukiwaniem keywordowym (opartym o słowa kluczowe, BM25), co pozwala na lepsze dopasowanie wyników wyszukiwania.

## Import wymaganych bibliotek

In [ ]:
import os
import glob
from typing import List
import weaviate
from weaviate.connect import ConnectionParams
from weaviate.classes.init import AdditionalConfig, Timeout
import weaviate.classes as wvc
from pydantic import BaseModel, Field
from tqdm import tqdm
from enum import Enum
from openai import OpenAI
import glob
import pickle

## Połączenie z API Open AI

In [ ]:
api_key = os.getenv('OPENAI_API_KEY')
openai_client = OpenAI(api_key=api_key)

## Definicja modeli Pydantic

Użyjemy Pydantic do definicji schematów dla ekstrakcji sekcji ulotki.

In [ ]:
class DrugSectionType(str, Enum):
    """Typy sekcji w ulotkach leków"""
    SKŁAD = "SKŁAD"
    WSKAZANIA = "WSKAZANIA"
    DAWKOWANIE = "DAWKOWANIE"
    PRZECIWWSKAZANIA = "PRZECIWWSKAZANIA"
    ŚRODKI_OSTROŻNOŚCI = "ŚRODKI OSTROŻNOŚCI"
    DZIAŁANIA_NIEPOŻĄDANE = "DZIAŁANIA NIEPOŻĄDANE"
    PRZECHOWYWANIE = "PRZECHOWYWANIE"
    PRODUCENT = "PRODUCENT"
    CENA = "CENA"
    INTERAKCJE = "INTERAKCJE"
    INNE = "INNE"
    
    @classmethod
    def from_text(cls, text: str) -> 'DrugSectionType':
        """Konwertuje tekst na odpowiadający typ sekcji"""
        text = text.strip().upper()
        
        if "SKŁAD" in text:
            return cls.SKŁAD
        elif "WSKAZANIA" in text:
            return cls.WSKAZANIA
        elif "DAWKOWANIE" in text:
            return cls.DAWKOWANIE
        elif "PRZECIWWSKAZANIA" in text:
            return cls.PRZECIWWSKAZANIA
        elif "ŚRODKI_OSTROŻNOŚCI" in text:
            return cls.ŚRODKI_OSTROŻNOŚCI
        elif "DZIAŁANIA_NIEPOŻĄDANE" in text:
            return cls.DZIAŁANIA_NIEPOŻĄDANE
        elif "PRZECHOWYWANIE" in text:
            return cls.PRZECHOWYWANIE
        elif "PRODUCENT" in text:
            return cls.PRODUCENT
        elif "CENA" in text:
            return cls.CENA
        elif "INTERAKCJE" in text:
            return cls.INTERAKCJE
        else:
            return cls.INNE

In [ ]:
class DrugSection(BaseModel):
    """Model dla pojedynczej sekcji informacji o leku"""
    section_name: DrugSectionType = Field(..., description="Nazwa sekcji (SKŁAD, WSKAZANIA, DAWKOWANIE, PRZECIWWSKAZANIA, ŚRODKI_OSTROŻNOŚCI, DZIAŁANIA_NIEPOŻĄDANE, PRZECHOWYWANIE, PRODUCENT, CENA, INTERAKCJE, INNE)")
    content: str = Field(..., description="Treść sekcji zawierająca informacje o tym aspekcie leku")
    questions: List[str] = Field(..., description="Lista 2-30 pytań, na które odpowiada dana sekcja. Pytania muszą być różnorodne i dotyczyć różnych aspektów leku, i wymieniać lek z nazwy.")

class DrugLeaflet(BaseModel):
    """Model dla całej ulotki leku podzielonej na sekcje z pytaniami"""
    drug_name: str = Field(..., description="Pełna nazwa leku wraz z dawką i postacią")
    sections: List[DrugSection] = Field(..., description="Lista sekcji zawierających informacje o leku wraz z pytaniami")

## Funkcje do przetwarzania danych o lekach

In [ ]:
def read_drug_leaflet(file_path):
    """Odczytuje plik z ulotką leku"""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content

def extract_sections_and_questions_with_llm(content):
    """Ekstrahuje sekcje z ulotki leku i generuje pytania za pomocą GPT-4.1-mini (jedno zapytanie)"""
    # Define system and user prompts with triple quotes for better readability
    system_prompt = """
        Jesteś ekspertem w analizowaniu ulotek leków. Twoim zadaniem jest 
        wyekstrahować poszczególne sekcje z ulotki leku i dla każdej sekcji wygenerować zestaw pytań, 
        na które ta sekcja odpowiada. Upewnij się, że treść każdej sekcji jest kompletna i nie zawiera 
        elementów formatowania.
        
        Dla każdej sekcji wygeneruj 2-30 różnorodnych pytań, które mogą zadać 
        pacjenci lub lekarze. Pytania powinny być konkretne i wymieniać lek z nazwy, a najlepiej też odwoływać się 
        do jednego z faktów z treści, której dotyczą, np.:
        - Jakie są składniki leku X?
        - Czy lek x może być stosowany dłużej niż 3 dni?
        - Czy lek X można stosować w ciąży?
        - Czy lek X może powodować mdłości?
    """
    
    user_prompt = f"""
        Oto ulotka leku. Wyekstrahuj wszystkie sekcje, ich treść, oraz 
        stwórz dla każdej sekcji listę pytań, na które ta sekcja odpowiada:
        ---
        {content}
    """
    
    # Make the API call
    response = openai_client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        response_format=DrugLeaflet,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.4,
        max_tokens=32768
    )
    
    return response.choices[0].message.parsed

### Test generowania sekcji i pytań

In [ ]:
drug_files = glob.glob("008-02. RAG-data/drug-data/*.txt")

test_file = drug_files[0]
file_name = os.path.basename(test_file)

print(f"Testowanie na pliku: {file_name}")

# Odczytaj ulotkę leku
content = read_drug_leaflet(test_file)
print(f"Zawartość ulotki (pierwsze 200 znaków): {content[:200]}...\n")

# Ekstrahuj sekcje i generuj dla nich pytania za pomocą LLMa
print("Ekstrahowanie sekcji za pomocą LLMa...")
leaflet = extract_sections_and_questions_with_llm(content)

print(leaflet.model_dump_json(indent=4))

## Przetworzenie danych na format do załadowania do Weaviate

Funkcja przetwarzająca ulotki na dane do załadowania do bazy Weaviate

In [ ]:
# Inicjalizacja globalnych zmiennych
all_data_objects = []
processed_files = set()


def create_drug_info_object(drug_name, section_name, content, questions):
    """Tworzy obiekt informacji o leku do zapisania w Weaviate"""
    return {
        "drug_name": drug_name,
        "section_name": section_name,
        "content": content,
        "questions": questions
    }

def process_drug_leaflets_to_data_objects():
    """Przetwarza ulotki leków na data_objecty we właściwym formacie do załadowania do Weaviate.
    Pomija pliki, które zostały już przetworzone w poprzednich wywołaniach.
    Aktualizuje globalne zmienne all_data_objects i processed_files.
    """
    global all_data_objects, processed_files
    
    drug_files = glob.glob("008-02. RAG-data/drug-data/*.txt")
    files_to_process = [file for file in drug_files if file not in processed_files]
    
    if not files_to_process:
        print("Wszystkie pliki zostały już przetworzone.")
        return
    
    # Użycie tqdm do wizualizacji postępu
    for file_path in tqdm(files_to_process, desc="Przetwarzanie ulotek leków"):
        # Odczytaj ulotkę leku
        content = read_drug_leaflet(file_path)
        
        # Ekstrahuj sekcje
        leaflet = extract_sections_and_questions_with_llm(content)
        
        # Przetwarzanie każdej sekcji
        for section in tqdm(leaflet.sections, desc=f"Sekcje {leaflet.drug_name}", leave=False):
            # Utwórz obiekt danych
            data_object = create_drug_info_object(
                drug_name=leaflet.drug_name,
                section_name=section.section_name.value,
                content=section.content,
                questions=section.questions
            )
            
            # Dodajemy obiekt do listy
            all_data_objects.append(data_object)
        
        # Oznaczamy plik jako przetworzony
        processed_files.add(file_path)
    
    print(f"Zakończono przetwarzanie. Łącznie {len(all_data_objects)} obiektów danych w pamięci.")

In [ ]:
process_drug_leaflets_to_data_objects()

In [ ]:

all_data_objects

In [ ]:
with open("008-02. RAG-data/drug-data.pkl", "wb") as f:
    pickle.dump(all_data_objects, f)

In [ ]:
with open("008-02. RAG-data/drug-data.pkl", "rb") as f:
    all_data_objects = pickle.load(f)

## Połączenie z serwerem Weaviate

In [ ]:
# Określenie adresu serwera Weaviate
weaviate_host = "weaviate" # skonfigurowany w docker-compose
weaviate_port_http = 8080
weaviate_port_grpc = 50051

# Próba połączenia z serwerem Weaviate
weaviate_client = weaviate.WeaviateClient(
    connection_params=ConnectionParams.from_params(
        http_host=weaviate_host,
        http_port=weaviate_port_http,
        http_secure=False,  # Używamy nieszyfrowanego połączenia HTTP
        grpc_host=weaviate_host,
        grpc_port=weaviate_port_grpc,
        grpc_secure=False,  # Używamy nieszyfrowanego połączenia gRPC
    ),
    # Bez klucza API, ponieważ łączymy się do lokalnego serwera
    additional_config=AdditionalConfig(
        timeout=Timeout(init=30, query=60, insert=120),  # Wartości w sekundach
    ),
    skip_init_checks=False
)

# Jawne połączenie z serwerem
weaviate_client.connect()
print("Połączono z serwerem Weaviate.")

## Definicja schemy Weaviate dla danych lekarstw

Stworzymy klasę `DrugInfo` w Weaviate do przechowywania informacji o lekach.

In [ ]:
weaviate_client.collections.list_all()

In [ ]:
# usunięcie istniejącej kolekcji, jeśli istnieje
weaviate_client.collections.delete("DrugInfo")

In [ ]:
# Tworzenie kolekcji dla informacji o lekach
drug_info_collection = weaviate_client.collections.create(
    name="DrugInfo",
    description="Informacje o leku pochodzące z ulotki",
    vectorizer_config=weaviate.classes.config.Configure.Vectorizer.text2vec_transformers(),
    properties=[
        weaviate.classes.config.Property(
            name="drug_name",
            data_type=weaviate.classes.config.DataType.TEXT,
            description="Nazwa leku",
            skip_vectorization=True, # nie wektoryzujemy nazwy leku
            index_filterable=True,
            index_searchable=True,
            tokenization=weaviate.classes.config.Tokenization.TRIGRAM # tokenizacja w 3-gramy, dzięki czemu możemy dopasować formy fleksyjne
            # Np. 'paracetamol' zostanie podzielony na: "par", "ara", "rac", "ace", "cet", "eta", "tam", "amo", "mol"
        ),
        weaviate.classes.config.Property(
            name="section_name",
            data_type=weaviate.classes.config.DataType.TEXT,
            description="Nazwa sekcji ulotki leku",
            skip_vectorization=False,
            index_filterable=True,
            index_searchable=True
        ),
        weaviate.classes.config.Property(
            name="content",
            data_type=weaviate.classes.config.DataType.TEXT,
            description="Treść sekcji",
            skip_vectorization=False,
            index_filterable=True,
            index_searchable=True
        ),
        weaviate.classes.config.Property(
            name="questions",
            data_type=weaviate.classes.config.DataType.TEXT_ARRAY,
            description="Pytania, na które odpowiada ten fragment",
            skip_vectorization=False,
            index_filterable=True,
            index_searchable=True
        )
    ],
    vector_index_config=weaviate.classes.config.Configure.VectorIndex.hnsw(
        distance_metric=weaviate.classes.config.VectorDistances.COSINE
    )
)
print("Pomyślnie utworzono kolekcję DrugInfo w Weaviate.")

## Ładowanie danych lekarstw do Weaviate

Funkcja ładująca wszystkie dane o lekach do Weaviate.

In [ ]:
def load_data_objects_to_weaviate(data_objects=None):
    """Ładuje data_objecty do Weaviate.
    
    Args:
        data_objects (List, optional): Lista data_objectów do załadowania.
    """

    collection = weaviate_client.collections.get("DrugInfo")
    
    # Ładowanie danych do Weaviate z użyciem dynamicznego batchingu
    with collection.batch.dynamic() as batch:
        for data_object in tqdm(data_objects, desc="Ładowanie danych do Weaviate"):
            # Dodajemy obiekt do batcha
            batch.add_object(properties=data_object)
    
    print(f"Zakończono ładowanie {len(data_objects)} obiektów danych do Weaviate.")

In [ ]:
load_data_objects_to_weaviate(all_data_objects)

## Funkcje do wyszukiwania w RAG-u hybrydowym

In [ ]:
def hybrid_search(query, limit=10):
    """Wykonuje hybrydowe wyszukiwanie w Weaviate
    
    Args:
        query: Zapytanie do wyszukania
        limit: Liczba wyników do zwrócenia
        
    Returns:
        Lista słowników zawierających znalezione obiekty
    """
    collection = weaviate_client.collections.get("DrugInfo")

    query_results = collection.query.hybrid(
        query=query,
        alpha=0.5,  # 50% wektory, 50% bm25 (słowa kluczowe)
        query_properties=["drug_name", "content", "questions"],
        return_metadata=wvc.query.MetadataQuery(score=True, explain_score=True),
        limit=limit
    )
    
    # Przekształć wyniki z formatu obiektowego na listę słowników
    formatted_results = []
    for obj in query_results.objects:
        formatted_results.append({
            'drug_name': obj.properties.get('drug_name', ''),
            'section_name': obj.properties.get('section_name', ''),
            'content': obj.properties.get('content', ''),
            'questions': obj.properties.get('questions', []),
            'score': obj.metadata.score,
            'explained_score': obj.metadata.explain_score,
        })
    
    return formatted_results

In [ ]:
hybrid_search("skutki uboczne Gastropril")

## Prosta funkcja opakowująca całość RAGa

Teraz stworzymy funkcję, która połączy wszystkie komponenty naszego hybrydowego systemu RAG. Funkcja ta pobierze zapytanie użytkownika, wykona wyszukiwanie hybrydowe w bazie Weaviate, a następnie użyje modelu językowego do wygenerowania odpowiedzi na podstawie znalezionych informacji o lekach.

In [ ]:
def ask_drug_rag(query, limit=5):
    """Zadaje pytanie do systemu RAG i wyświetla wyniki wraz z odpowiedzią wygenerowaną przez GPT"""
    results = hybrid_search(query, limit=limit)
    
    # Przygotowanie kontekstu dla GPT
    context = ""
    for i, result in enumerate(results, 1):
        context += f"\nŹródło {i}:\n"
        context += f"Lek: {result['drug_name']}\n"
        context += f"Sekcja: {result['section_name']}\n"
        context += f"Treść: {result['content']}\n"
    
    # Definicja promptów
    system_prompt = "Jesteś asystentem medycznym specjalizującym się w informacjach o lekach. " \
                    "Odpowiadaj zwięźle, rzeczowo i profesjonalnie, bazując wyłącznie na dostarczonych informacjach. " \
                    "Jeśli nie masz wystarczających danych, przyznaj to otwarcie."
    
    user_prompt = f"Na podstawie poniższych informacji o lekach, odpowiedz na pytanie: {query}\n\n{context}"
    
    # Generowanie odpowiedzi przez GPT-4.1-mini
    response = openai_client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.1,
        max_tokens=5000
    )
    
    return response.choices[0].message.content

In [ ]:
# Przykładowe użycie interfejsu
r = ask_drug_rag("Jakie są działania niepożądane Gastroprilu? Jak dorośli powinni go dawkować?", limit=20)
print(r)

In [ ]:
r = ask_drug_rag("Powiedz mi wszysko co wiesz o Gastroprilu", limit=20)
print(r)

In [ ]:
weaviate_client.close()